In [1]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import tensorflow as tf
from baselines.ppo1 import cnn_policy_particle_sweep_mult_density, cnn_policy_particle_sweep_mult_density_share_conv
import baselines.common.tf_util as U
import gym
import pandas as pd
%matplotlib inline

# Load Saved Trajectory

In [2]:
traj_data = joblib.load("data/trajs/plasticOneClusterBarCentric.pkl")
observations = traj_data["observations"]
observations = np.array(observations)
actions = traj_data["actions"]
actions = np.array(actions)

print("Observation Shape: ",observations.shape)
print("Action Shape: ", actions.shape)

#print("Split observations into separate scenes")
obs_at_frame = observations[0,0]


Observation Shape:  (1000, 49, 3077)
Action Shape:  (1000, 49, 5)


# Load Saved Policy

In [3]:
def restore_policy(sess, policy, policy_params):
    cur_scope = policy.get_variables()[0].name[0:policy.get_variables()[0].name.find('/')]
    orig_scope = list(policy_params.keys())[0][0:list(policy_params.keys())[0].find('/')]

    for i in range(len(policy.get_variables())):
        assign_op = policy.get_variables()[i].assign(
            policy_params[policy.get_variables()[i].name.replace(cur_scope, orig_scope, 1)])
        sess.run(assign_op)

def cnn_policy_fn(name, ob_space, ac_space):  # pylint: disable=W0613
    return cnn_policy_particle_sweep_mult_density.CnnPolicyParticleSweepMultiDensity(name=name, ob_space=ob_space,
                                                                             ac_space=ac_space
                                                                             )

def cnn_policy_shared_val_fn(name, ob_space, ac_space):  # pylint: disable=W0613
    return cnn_policy_particle_sweep_mult_density_share_conv.CnnPolicyParticleSweepMultiDensityShareConv(name=name, ob_space=ob_space,
                                                                             ac_space=ac_space
                                                                             )

# saliencies = saliencies[0,0]

# Draw Saliency Maps

In [4]:
value_saliencies= None
policy_saliencies = None
def get_saliency_from_policy(filename,fn):
#     filename
#     policy_filename = "data/trajs/policies/"
#     policy_filename +="singleClusterFixed/policy_params_370.pkl"
    # print(policy_filename)
    env = gym.make("FlexPlasticReshaping-v5")
    policy_param =joblib.load(filename)

    tf.reset_default_graph()
    activation1 = None
    activation2 = None

    saliencies = None
    with U.single_threaded_session() as sess:
        pi = fn('pi', env.observation_space, env.action_space)
        restore_policy(sess, pi, policy_param)

    #     tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='pi/pol/conv_1')
        # curr_particle_density_diff = obs_at_frame[15:15+32*32]
        # curr_particle_density_diff = np.reshape(curr_particle_density_diff,(32,32))
        # prev_particle_density_diff = obs_at_frame[15+32*32::]
        # prev_particle_density_diff = np.reshape(prev_particle_density_diff,(32,32))
        # conv_obs = np.concatenate([curr_particle_density_diff,prev_particle_density_diff],axis=-1)
    #     for i in range(1000):
        valSal = np.array(pi.get_value_saliency(observations[:,0]))[0,0]

        polSal = np.array(pi.get_policy_saliency(observations[:,0]))[0,0]
    return valSal,polSal


def saliency_at_frame(i):
    bar_obs = 5
    pol_sal = policy_saliencies[i]
    val_sal = value_saliencies[i]
    obs = observations[i,0]
    
    density_pol_saliency = pol_sal[bar_obs:bar_obs+32*32]
    density_pol_saliency = np.reshape(density_pol_saliency,(32,32))
#     density_pol_saliency -= np.min(density_pol_saliency)

    goal_pol_saliency = pol_sal[bar_obs+32*32:bar_obs+2*32*32]
    goal_pol_saliency = np.reshape(goal_pol_saliency,(32,32))
#     goal_pol_saliency -= np.min(goal_pol_saliency)
    
    
    bar_vel_pol_saliency = pol_sal[:bar_obs]
    bar_vel_pol_saliency = np.expand_dims(bar_vel_pol_saliency,axis=0)
    
    density_val_saliency = val_sal[bar_obs:bar_obs+32*32]
    density_val_saliency = np.reshape(density_val_saliency,(32,32))
#     density_val_saliency -= np.min(density_val_saliency)
    
    goal_val_saliency = val_sal[bar_obs+32*32:bar_obs+2*32*32]
    goal_val_saliency = np.reshape(goal_val_saliency,(32,32))
#     goal_val_saliency -= np.min(goal_val_saliency)
    
    bar_vel_val_saliency = val_sal[:bar_obs]
    bar_vel_val_saliency = np.expand_dims(bar_vel_val_saliency,axis=0)
    
    density_obs = obs[bar_obs:bar_obs+32*32]
    density_obs = np.reshape(density_obs,(32,32))
    
    goal_obs = obs[bar_obs+32*32:bar_obs+2*32*32]
    goal_obs= np.reshape(goal_obs,(32,32))
    
    bar_vel_obs = obs[:bar_obs]
    bar_vel_obs = np.expand_dims(bar_vel_obs,axis=0)

    fig, axes = plt.subplots(3,3,figsize=(10,10))
    axes[0,0].imshow(density_pol_saliency)
    axes[0,1].imshow(goal_pol_saliency)
    img1 = axes[0,2].imshow(bar_vel_pol_saliency)

    axes[1,0].imshow(density_val_saliency)
    axes[1,1].imshow(goal_val_saliency)
    img1 = axes[1,2].imshow(bar_vel_val_saliency)
    
    
    axes[2,0].imshow(density_obs,  vmin=0)
    axes[2,1].imshow(goal_obs,  vmin=0)
    img2 = axes[2,2].imshow(bar_vel_obs)
    
    fig.tight_layout()
    

In [5]:
policy_filename = "data/trajs/policies/"
policy_filename +="policy_params_360.pkl"
value_saliencies,policy_saliencies = get_saliency_from_policy(policy_filename,cnn_policy_shared_val_fn)
    
#     print (pd.DataFrame(density_saliency))
w = widgets.IntSlider(min=0,max=999)

# plot_at_frame(0,fig,axes)
interact(saliency_at_frame, i = w)



[2019-11-20 10:20:57,015] Making new env: FlexPlasticReshaping-v5


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/yzhang/MyDartEnv/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


pyFlex initialization OK
image 0 Conv input Shape:  (?, 32, 32, 1)
Instructions for updating:
Colocations handled automatically by placer.


[2019-11-20 10:20:57,943] From /home/yzhang/anaconda3/envs/deformableManip/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


image 0 Conv1 out Shape:  (?, 32, 32, 2)
image 0 Conv2 out Shape:  (?, 16, 16, 2)
image 0 Pool2 out Shape:  (?, 8, 8, 2)
Instructions for updating:
Use keras.layers.dense instead.


[2019-11-20 10:20:57,973] From /home/yzhang/MyBaselines/baselines/ppo1/cnn_policy_particle_sweep_mult_density_share_conv.py:90: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


image 0 Fully Connected Output (?, 32)
image 1 Conv input Shape:  (?, 32, 32, 1)
image 1 Conv1 out Shape:  (?, 32, 32, 2)
image 1 Conv2 out Shape:  (?, 16, 16, 2)
image 1 Pool2 out Shape:  (?, 8, 8, 2)
image 1 Fully Connected Output (?, 32)
image 2 Conv input Shape:  (?, 32, 32, 1)
image 2 Conv1 out Shape:  (?, 32, 32, 2)
image 2 Conv2 out Shape:  (?, 16, 16, 2)
image 2 Pool2 out Shape:  (?, 8, 8, 2)
image 2 Fully Connected Output (?, 32)
First FC Shape:  (?, 101)
FC Shape:  (?, 64)
FC Shape:  (?, 64)
First FC Shape:  (?, 101)
FC Shape:  (?, 64)
FC Shape:  (?, 64)


interactive(children=(IntSlider(value=0, description='i', max=999), Output()), _dom_classes=('widget-interact'…

<function __main__.saliency_at_frame(i)>